# Install some packages

packages installed from within notebook seems to not persist when server shutdown

conda install was too slow solving environment, didn't bother

In [1]:
! pip install spacy
! python -m spacy download en_core_web_sm
! pip install wordcloud
nltk.download('brown')

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


NameError: name 'nltk' is not defined

# Just some imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from pprint import pprint
import nltk
from nltk.corpus import words, brown
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import boto3 

# All functions are here

Not the best readiblity for notebook, since you have to jump back and forth to read function definitions

But it's the best for clarity and later refactoring/producitonising

In [ ]:
def re_find_all_start_positions(regex_pat: str, text: str) -> list:
    matches = list( re.finditer(regex_pat, text, flags=re.MULTILINE) )
    start_positions = [m.start() for m in matches]
    return start_positions
    
def re_split(regex_pat: str, text: str) -> list:
    """Custom regex split solution, re.split() either throw away the delimiter 
    or keep it as separate items, doesn't suit the needs here.
    """
    split_positions = re_find_all_start_positions(regex_pat, text)
    if split_positions[0] > 0:  # add starting position if it's not there
        split_positions = [0] + split_positions
    split_positions = split_positions + [len(text)]  # add end position

    splits = []
    n_splits = len(split_positions) - 1
    for i in range(n_splits):
        splits.append(text[split_positions[i] : split_positions[i+1]])
    
    assert(''.join(splits) == text)
    
    return splits

def strip_scene_header(scene: str) -> str:
    return re.sub(r"^\tHAMLET\n+ACT [A-Z]+\n+SCENE [A-Z]+\t.+\n+", "", scene, flags=re.MULTILINE)

def re_search(pat: str, text: str) -> str:
    """re.search() wrapper
    returns first matching group, with MULTILINE flag
    returns N/A if not found"""
    match = re.search(pat, text, flags=re.MULTILINE)
    if match:
        return match.group(1)
    else:
        return "N/A"

def get_act_name(scene_header: str) -> str:
    return re_search(r"^\tHAMLET\n+(ACT [A-Z]+)(\n+)", scene_header)
    
def get_scene_name(scene_header: str) -> str:
    return re_search(r"(^SCENE [A-Z]+)(?:\t)", scene_header)
    
def get_scene_title(scene_header: str) -> str:
    return re_search(r"(?:^SCENE [A-Z]+\t)(.+$)", scene_header)
    
def get_scenes_df(full_text: str) -> pd.DataFrame:
    scenes = re_split(r"^\tHAMLET\n+ACT [A-Z]+\n+SCENE [A-Z]+\t.+\n+", full_text)
    
    scenes_dict_list = []
    for s in scenes:
        act_name = get_act_name(s)
        scene_name = get_scene_name(s)
        scene_title = get_scene_title(s)
        scene_text = s
        scenes_dict_list.append({'act_name': act_name, 'scene_name': scene_name, 
                                'scene_title': scene_title, 'scene_text': scene_text})
    
    scenes_df = pd.DataFrame(scenes_dict_list)
    assert(''.join(scenes_df.scene_text) == full_text)
    return scenes_df

def get_scene_header(scene: str) -> str:
    return re_search(r"(^\tHAMLET\n+ACT [A-Z]+\n+SCENE [A-Z]+\t.+\n+)", scene)

def split_scene_into_paragraphs(scene_df_row: pd.Series) -> pd.DataFrame:
    scene = scene_df_row.scene_text
    scene_header = get_scene_header(scene)
    
    paragraphs = []    
    if scene_df_row.scene_name == 'N/A':  # cast section at the beginning of the book
        paragraphs.append({'paragraph_type': 'cast', 'paragraph_text': scene})
    else:  # a proper scene
        scene_body = strip_scene_header(scene)
        header_paragraph = {'paragraph_type': 'header', 'paragraph_text': scene_header}
        paragraphs.append(header_paragraph)
        
        re_speech_header = r"(?<=\n\n)[A-Z][A-Za-z ]+\t"
        re_action_paragraph = r"^\s*\[.+$"

        for paragraph_text in re_split(re_speech_header, scene_body):
            if re.match(re_action_paragraph, paragraph_text, flags=re.MULTILINE):
                paragraph_type = 'action'
            else:
                paragraph_type = 'speech'
            paragraph_dict = {'paragraph_type': paragraph_type, 'paragraph_text': paragraph_text}
            paragraphs.append(paragraph_dict)
    
    # convert to dataframe and copy act/scene info from scene dataframe
    paragraphs_df = pd.DataFrame(paragraphs)
    scene_info = scene_df_row.drop('scene_text')
    for col, val in scene_info.iteritems():
        paragraphs_df[col] = val
    
    assert( ''.join(paragraphs_df.paragraph_text) == scene_df_row.scene_text )
    return paragraphs_df

def get_paragraphs_df(scenes_df: pd.DataFrame) -> pd.DataFrame: 
    paragraphs_df = pd.concat([ split_scene_into_paragraphs(row) for i, row in scenes_df.iterrows() ])
    paragraphs_df = paragraphs_df.reset_index(drop=True)  # reset index to [0,1,2,3,....]
    paragraphs_df = paragraphs_df.reset_index().rename(columns={'index': 'paragraph_id'})  # convert index to paragraph_id
    paragraphs_df['characters'] = paragraphs_df.paragraph_text.map(get_characters_in_paragraph)
    return paragraphs_df

def get_characters_in_paragraph(paragraph_text: str) -> list:
    chars = re.findall(r"(?!^SCENE.+)(^[A-Z][A-Za-z ]+)(?:\t)", paragraph_text, flags=re.MULTILINE)
    return chars

def split_paragraph_into_lines(paragraph_df_row: pd.Series) -> pd.DataFrame:
    paragraph_text = paragraph_df_row.paragraph_text
    lines = re_split(r"^.*\n", paragraph_text)
    
    re_empty_line = r"(^[A-Z][A-Za-z ]+\t[\|\s]*$)|(^[\s]*$)"  # also match 'CHARACTER    |' as empty line
    re_character_name = r"^[A-Z][A-Za-z ]+\t"
    re_action_line = r"^\s*\[.*\]\s*$"
    
    lines_dict_list = []
    for line in lines:
        if re.match(re_empty_line, line, flags=re.MULTILINE):
            line_type = 'empty'
        elif paragraph_df_row.paragraph_type == 'cast':
            line_type = 'cast'
        elif paragraph_df_row.paragraph_type == 'header':
            line_type = 'header'
        elif paragraph_df_row.paragraph_type == 'action':
            line_type = 'action'
        elif re.match(re_action_line, line, flags=re.MULTILINE):
            line_type = 'action'
        else:
            line_type = 'speech'
        line_dict = {'line_type': line_type, 'line': line}
        lines_dict_list.append(line_dict)
    lines_df = pd.DataFrame(lines_dict_list)
    # copy paragraph info over except paragraph_text
    paragraph_info = paragraph_df_row.drop('paragraph_text')
    for col, val in paragraph_info.iteritems():
        lines_df[col] = [val] * len(lines_df)  # had to use this special pattern since character column contains list
    
    assert(''.join(lines_df.line) == paragraph_df_row.paragraph_text)
    return lines_df

def get_lines_df(paragraphs_df: pd.DataFrame) -> pd.DataFrame: 
    lines_df = pd.concat([ split_paragraph_into_lines(row) for i, row in paragraphs_df.iterrows() ])
    lines_df = lines_df.reset_index(drop=True)  # reset index to [0,1,2,3,....]
    lines_df = lines_df.reset_index().rename(columns={'index': 'line_id'})  # convert index to line_id
    return lines_df

def spacy_lemmatise(text: str) -> list:
    # tried to lemmatise the tokens to use it with a smaller corpus (nltk's words.words())
    # turned out it's better to simply use a bigger corpus
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    doc = nlp(text)
    lemmas = [token.lemma_.lower() for token in doc if token.is_alpha]
    tokens = [token.text.lower() for token in doc if token.is_alpha]
    context = []
    for i in range(len(tokens)):
        i_start = i-5 if i>=5 else 0
        i_end = i+5 if i<=len(tokens)-5 else len(tokens)
        context.append(' '.join([t for t in tokens[i_start:i_end]]))
    lemma_map = pd.DataFrame({'token': tokens, 'lemma': lemmas, 'context': context}).drop_duplicates()
    return tokens, lemmas, lemma_map

def get_common_english_words(topn: int) -> list:
    return pd.Series([w.lower() for w in brown.words()])\
            [lambda s: s.str.isalpha()]\
            .value_counts().head(topn).index.values.tolist()

def show_word_cloud(text: str):
    wordcloud = WordCloud(width=800, height=400).generate(text)
    fig, ax = plt.subplots(figsize=(15,8))
    _ = plt.imshow(wordcloud, interpolation='bilinear')
    _ = plt.axis("off")
    
def detect_sentiment_batch(texts: list) -> tuple:
    comprehend = boto3.client('comprehend', region_name='ap-southeast-2')
    results = comprehend.batch_detect_sentiment(TextList=texts, LanguageCode='en')
    sentiments = [ r['Sentiment'] for r in results['ResultList'] ]
    positive_scores = [ r['SentimentScore']['Positive'] for r in results['ResultList'] ]
    negative_scores = [ r['SentimentScore']['Negative'] for r in results['ResultList'] ]
    neutral_scores = [ r['SentimentScore']['Neutral'] for r in results['ResultList'] ]
    mixed_scores = [ r['SentimentScore']['Mixed'] for r in results['ResultList'] ]
    return sentiments, positive_scores, negative_scores, neutral_scores, mixed_scores

def batch_sentiment(texts: list) -> list:
    max_batch_size = 25  # Comprehend API batch size limit
    sentiments = []
    for i in range(0, len(texts), max_batch_size):
        batch_texts = texts[i: i + max_batch_size]
        # made it return the individual scores but didn't end up using them
        batch_sentiments, _, _, _, _ = detect_sentiment_batch(batch_texts)
        sentiments = sentiments + batch_sentiments
        print(f'Processed {len(sentiments)} texts')
    return sentiments

# Parameters

Thought there would be more but ended up a single one...

Some of the stuff should be paramters though if we want to reuse/productionise like the regex patterns

They were actually in here before I decided to move them into functions for better readability

In [ ]:
DATA_FILE = "hamlet.txt"

# "main" routines starts here

## Read file

In [ ]:
full_book = open(DATA_FILE).read()

full_book[:300]

## Convert into Scenes dataframe

Each Scene would be a row, and I decided to do a loss-less split, in a sense that if you simply concatenate all scene_text, you will end up getting the exact document before split. 

This was inspired by spacy's non-destructive tokenisation, I always admired their approach and this is an opportunity to do something similar. 

In [ ]:
scenes_df = get_scenes_df(full_book)

scenes_df

## Further split into paragraphs

Again the splits are loss less

In [ ]:
paragraphs_df = get_paragraphs_df(scenes_df)

paragraphs_df.head()

## Split into lines

The document eventually became a denormalised data table (OLAP's favourite :) )

Note each line is tagged as header/speech/action/empty etc.

In [ ]:
lines_df = get_lines_df(paragraphs_df)

In [ ]:
lines_df.iloc[410:420, :]

### Verify that it's still a loss-less split end to end


In [ ]:
''.join(lines_df.line) == full_book

# Get character lines

One of the tricky things was multiple characters may say the same line together, it's taken care of during the paragraph parsing

In [ ]:
speech_lines = lines_df[lambda df: df.line_type=='speech'][['characters', 'line', 'line_id']]

character_lines = pd.DataFrame()
for i, line in speech_lines.iterrows():
    for char in line.characters:
        row = line.to_dict()
        row.pop('characters')
        row['character'] = char
        character_lines = character_lines.append(row, ignore_index=True)

In [ ]:
character_lines.head()

## Number of lines per character

With all the hard work above, this is now very simple. 

This version seems to have fewer lines comparing to the numbers listed on Wikipedia?

In [ ]:
character_lines.groupby('character').size().sort_values(ascending=False)

### Get all the non empty lines

In [ ]:
non_empty_lines = ''.join(lines_df[lambda df: df.line_type!='empty'].line)

### Use spacy for tokenising

Tried lemmatising, turned out it wasn't necessary with a bigger dictionary

In [ ]:
tokens, tokens_lemmatised, lemma_map = spacy_lemmatise(non_empty_lines)

### Get a list of common enligsh words
nltk's Brown corpus was created in 1961, seems to be a good choice for older literature

In [ ]:
common_english_words = get_common_english_words(10000)

common_english_words[:10]

### Get a list of character names

to be excluded in the word cloud

some characters were referred to without titles, hence the added list

In [ ]:
char_names = character_lines.character.unique().tolist() + ['CLAUDIUS', 'POLONIUS', 'GERTRUDE', 'FORTINBRAS']
char_names = [c.lower() for c in char_names]

### Here are the unusal tokens (words)

Convert to Series since list comprehension was too slow

In [ ]:
tokens_unusual = pd.Series(tokens)\
                [lambda s: ~s.isin(common_english_words + char_names)]  

tokens_unusual.value_counts()

## Unusual word cloud

In [ ]:
show_word_cloud(' '.join(tokens_unusual))

# Get sentiments using AWS Comprehend

Using a top-tier paid NLP API seems cheating, but realistically the next best free option is probably fine-tuning BERT with a public dataset. But tuning BERT itself is costly and a good public sentiment dataset is hard to find (especially for old english literature). The only off the shelf options that can be done within a few hours are probably nltk and TextBlob. However they are mostly rule based, and work even worse with longer text. 

In [ ]:
# AWS Comprehend's sentiment API has a limit of 5KB per document
paragraphs_df.paragraph_text.map(len).max()

In [ ]:
scene_paragraphs = paragraphs_df[lambda df: df.act_name!='N/A'].copy() # exclude the cast section at the start

In [ ]:
sentiments = batch_sentiment(scene_paragraphs.paragraph_text.tolist())

In [ ]:
scene_paragraphs['sentiment'] = sentiments

## Number of paragraphs by sentiment type per scene

In [ ]:
sentiment_by_scene = scene_paragraphs.\
                        pivot_table(index=['act_name', 'scene_name'], columns='sentiment', aggfunc='size')\
                        .fillna(0)

sentiment_by_scene

### Sentiment Ratios per Scene

In [ ]:
sentiment_by_scene.pipe(lambda df: df.divide(df.sum(axis=1), axis=0) )

# Sentiment per Scene Visualised

The last scene seems pretty tragic..

In [ ]:
sentiment_by_scene.pipe(lambda df: df.divide(df.sum(axis=1), axis=0) )\
.plot.bar(stacked=True, figsize=(14,7))